#1- Import Necessary modules and packages


In [1]:
%%capture
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

!pip install pretty_midi
!pip install pydub

In [2]:
%%capture

import pickle
import cv2
import fluidsynth
import pretty_midi
import numpy as np

from IPython.display import Audio, HTML
from moviepy.editor import *
from music21 import note, instrument, chord, stream
from tensorflow.keras.models import load_model
from pydub import AudioSegment
from IPython.display import HTML
from base64 import b64encode

#2- Load the Saved Models
Load the trained models. 

lrcn_model : Model trained on UCF-50 dataset to recognize the activity happeining in a video.

music_model : Model to generate never before piano music. 

In [3]:
lrcn_model = load_model('/content/drive/MyDrive/project/autofoley/video_lrcn/lrcn_model.h5')
music_model = load_model('/content/drive/MyDrive/project/autofoley/music/trained_model/music_model.h5')

#3- Video Activity Recognition 

In [4]:
def predict_single_action(model, video_file_path):
    '''
    This will predict a single action using the model that is being performed in the video. 
    It is same function used while creating the lrcn_model. We used it there for testing purpose. 
    Args:
        model : The model to be used for making predictions . 
        video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    '''

    # Initialize the video_reader. 
    video_reader = cv2.VideoCapture(video_file_path)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames_list = []

    # Get the number of frames in the video, and raise Error if not enough frames in the video .
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video_reader.get(cv2.CAP_PROP_FPS)

    if video_frames_count < VIDEO_SEQUENCE_LENGTH:
        print("ERROR : Not Enough Frames to make Predictions")
        return 

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/VIDEO_SEQUENCE_LENGTH),1)

    for frame_counter in range(VIDEO_SEQUENCE_LENGTH):

        #Set the video_reader to the desired frame. 
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read() 

        # Check if frame is not read properly then break the loop.
        if not success:
            print("Video Reading Failed")
            break

        # Resize, normalize and append
        resized_frame = cv2.resize(frame, (VIDEO_IMAGE_HEIGHT, VIDEO_IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)

    #Make prediction 
    predicted_labels_probabilities = model.predict(np.expand_dims(frames_list, axis = 0))[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    
    # Print Result
    print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label] * 100}')
        
    # Release Resources 
    video_reader.release()
    return predicted_label, round(video_frames_count/fps)

## 3.1 Load the saved helper values(same as used for training)

In [5]:
with open('/content/drive/MyDrive/project/autofoley/video_lrcn/lrcn_helpers', 'rb') as filepath:
    lrcn_helpers = pickle.load(filepath)
VIDEO_SEQUENCE_LENGTH = lrcn_helpers[0]
VIDEO_IMAGE_HEIGHT, VIDEO_IMAGE_WIDTH = lrcn_helpers[1],lrcn_helpers[2]
CLASSES_LIST = lrcn_helpers[3]

##3.2 Make Prediction

In [6]:
input_video_file_path = f'/content/drive/MyDrive/project/autofoley/video_lrcn/v_PlayingPiano_g06_c04.mp4'

# Perform Single Prediction on the Test Video.
predicted_label,video_time_duration = predict_single_action(lrcn_model, input_video_file_path)

Action Predicted: PlayingPiano
Confidence: 99.92813467979431


##3.3 Display Video

In [7]:
video_path = f'{input_video_file_path}'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

#4- Adding Generated Background Music

Although the lrcn_model is currently capable of recognizing [HorseRace, PlayingPiano, Swing, Tai-chi] classes, but at current we only have the model to add the background music iff predicted class is PlayingPiano. 
Generating background music for other classes would be very same, if we train the model on desired dataset.  

In [8]:
def generate_music_notes(model, how_many_notes, network_input, pitch_names, n_vocab):
    int_to_note_dict = dict((counter, note) for counter, note in enumerate(pitch_names))

    random_start_point = np.random.randint(0, len(network_input)-1)

    pattern = network_input[random_start_point]
    generated_notes = []

    for notes_counter in range(how_many_notes):
        reshaped_input = np.reshape(pattern, (1, len(pattern), 1))
        normalized_input = reshaped_input / float(n_vocab)

        predicted_next_seq = np.argmax(model.predict(normalized_input, verbose = 0))
        # predicted_next_seq = np.random.choice(Y_train.shape[1], 1, p=model.predict(normalized_input, verbose = 0)[0])[0]
        predicted_next_note = int_to_note_dict[predicted_next_seq]

        generated_notes.append(predicted_next_note)

        pattern = pattern[1:]
        pattern.append(predicted_next_seq)
    
    return generated_notes

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

_SAMPLING_RATE = 44100
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display(Audio(waveform_short, rate=_SAMPLING_RATE))

##4.1 Import music helpers (saved during training)

In [9]:
with open('/content/drive/MyDrive/project/autofoley/music/music_helpers', 'rb') as filepath:
    music_helpers = pickle.load(filepath)

MUSIC_SEQUENCE_LENGTH = music_helpers[0]
network_input = music_helpers[1]
loaded_midi_notes = music_helpers[2] 

music_vocab = set(loaded_midi_notes)
pitch_names = sorted(music_vocab)
n_vocab = len(music_vocab)

##4.2 Add Audio to Video

In [13]:
def add_audio_to_video(video_path, predicted_label):
    if predicted_label != 0:            #class at 0 is playingPiano in CLASS_LIST 
        print("Sorry Currently Audio Generation is only Available for playingPiano class")
        return
    
    #Generate Music using the music_model and create midi file
    generated_music_note = generate_music_notes(music_model,5*video_time_duration, network_input, pitch_names, n_vocab)
    create_midi(generated_music_note)

    #Convert .mid file to .wav file , and then .wav file to .mp3
    !fluidsynth -ni font.sf2 test_output.mid -F output.wav -r _SAMPLING_RATE
    AudioSegment.from_wav("output.wav").export("final_audio.mp3", format="mp3")

    #Add the generated music file to the video
    videoclip = VideoFileClip(f"{video_path}")
    audioclip = AudioFileClip("final_audio.mp3").set_duration(video_time_duration)

    new_audioclip = CompositeAudioClip([audioclip])
    videoclip.audio = new_audioclip
    videoclip.write_videofile("new_filename.mp4")

In [14]:
%%capture
add_audio_to_video(input_video_file_path, predicted_label)

  UserWarning)



In [15]:
video_file = f'new_filename.mp4'
mp4 = open(video_file,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width= 400 controls><source src={decoded_vid} type="video/mp4"></video>')